In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Data Import

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
parsedXML = ET.parse("/content/drive/My Drive/Masterarbeit/Data/Original/SEMEVAL-14/Laptops_Test_Gold.xml")

In [4]:
aspect_number = 7

In [5]:
dfcols = ['id','text']
for ii in range(1,aspect_number):
    dfcols.append("aspect_term_{}".format(ii))
    dfcols.append("aspect_polarity_{}".format(ii))
    dfcols.append("aspect_from_{}".format(ii))
    dfcols.append("aspect_to_{}".format(ii))          
df = pd.DataFrame(columns=dfcols)

for sentence in parsedXML.getroot():
    id = sentence.attrib.get('id')
    text = sentence.find('text').text
    line = [id,text]

    for asp in sentence.iter('aspectTerm'):
        term = asp.attrib.get("term")
        pol = asp.attrib.get("polarity")
        a_from = asp.attrib.get("from")
        a_to = asp.attrib.get("to")

        line += [term, pol, a_from, a_to]

    if len(line) < len(dfcols):
        pads = [None] * (len(dfcols)-len(line))
        line += pads
    
    df = df.append(pd.Series(line, index=dfcols), ignore_index=True)

In [6]:
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6
0,892:1,"Boot time is super fast, around anywhere from ...",Boot time,positive,0,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1144:1,tech support would not fix the problem unless ...,tech support,negative,0,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,805:2,but in resume this computer rocks!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,359:1,Set up was easy.,Set up,positive,0,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,562:1,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,22,31,touchscreen functions,negative,36,57,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,256:1,This hardware seems to be better than the iMac...,hardware,positive,5,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
796,246:1,I'm done with WinDoze computers.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
797,520:1,I've had it for about 2 months now and found n...,software,neutral,60,68,updates,neutral,72,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
798,306:2,the latest version does not have a disc drive.,disc drive,neutral,35,45,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# Drop duplicates

In [7]:
df[df.duplicated("text",keep=False)]

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6


In [8]:
text_counts = df.text.value_counts()
text_counts[:10]

I will not hesitate to recommend it to family and friends                                                                                 1
Great purchase and I definitely didn't miss out on the Black Friday deals.                                                                1
Now that I have upgraded to Lion I am much happier about MAC OS and have just bought an iMac for office.                                  1
Great deal on an amazing lap top!                                                                                                         1
This was absolutely annoying!                                                                                                             1
Let me start off by saying that I was highly reluctant to spend so much money on a laptop.                                                1
However, the experience was great since the OS does not become unstable and the application will simply shutdown and reopen.              1
I saw the bad review

# First Descriptive Analysis


## Aspects per sentence

In [9]:
pol_cols = ["aspect_polarity_"+str(ii) for ii in range(1,aspect_number)]

In [10]:
prev = 0
total_asp = 0
for no, col in enumerate(pol_cols):
    #print("sentences with at least ", no+1, "aspects:", sum(df[col].value_counts()))

    if col != "aspect_polarity_1":
        print("sentences with exactly ", no, "aspects:", prev - sum(df[col].value_counts()))
        total_asp += no * (prev - sum(df[col].value_counts()))

    if col == "aspect_polarity_"+str(aspect_number-1):
        print("sentences with exactly ", no+1, "aspects:", sum(df[col].value_counts()))
        total_asp += (no+1) * sum(df[col].value_counts())


    prev = sum(df[col].value_counts())

print("total no of aspects: ", total_asp)

sentences with exactly  1 aspects: 266
sentences with exactly  2 aspects: 105
sentences with exactly  3 aspects: 34
sentences with exactly  4 aspects: 10
sentences with exactly  5 aspects: 6
sentences with exactly  6 aspects: 1
total no of aspects:  654


## Sentiment Frequency

In [11]:
df_pol = df.loc[:,pol_cols]
df_pol_counts = df_pol.apply(pd.Series.value_counts)
df_pol_counts.sum(axis=1)

conflict     16.0
negative    128.0
neutral     169.0
positive    341.0
dtype: float64

In [12]:
sum(df_pol_counts.sum(axis=1))

654.0

## Sentences with more than one aspect

In [13]:
multi_counter = 0
for line in df.index:
    sentiment_list = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            sentiment_list += [df.loc[line,col]]
    if len(set(sentiment_list)) > 1:
        multi_counter += 1
multi_counter

40

# Remove "conflict"

In [14]:
for line in range(len(df)):
    for col in pol_cols:
        if df.loc[line,col] == "conflict":
            df.loc[line,col] = None
            number = col[-1:]
            df.loc[line,"aspect_term_"+str(number)] = None
            df.loc[line,"aspect_to_"+str(number)] = None
            df.loc[line,"aspect_from_"+str(number)] = None

In [15]:
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6
0,892:1,"Boot time is super fast, around anywhere from ...",Boot time,positive,0,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1144:1,tech support would not fix the problem unless ...,tech support,negative,0,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,805:2,but in resume this computer rocks!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,359:1,Set up was easy.,Set up,positive,0,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,562:1,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,22,31,touchscreen functions,negative,36,57,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,256:1,This hardware seems to be better than the iMac...,hardware,positive,5,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
796,246:1,I'm done with WinDoze computers.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
797,520:1,I've had it for about 2 months now and found n...,software,neutral,60,68,updates,neutral,72,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
798,306:2,the latest version does not have a disc drive.,disc drive,neutral,35,45,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# Check for wrong positions

In [16]:
mistakes = []
for ii in range(len(df)):
    for xx in range(1,aspect_number):
        asp_col = "aspect_term_"+str(xx)
        from_col = "aspect_from_"+str(xx)
        to_col = "aspect_to_"+str(xx)
        actual_term = df.loc[ii,asp_col]
        if actual_term != None:
            pos_term = df.text[ii][int(df.loc[ii,from_col]):int(df.loc[ii,to_col])]
            if actual_term != pos_term:
                mistakes += [ii]
                print(actual_term, pos_term)
mistakes

[]

# Check for wrong aspect terms

In [17]:
import nltk

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
for ii in range(len(df)):
    tokens = nltk.word_tokenize(df.text[ii])
    for xx in range(1,aspect_number):
        actual_term = df.loc[ii,"aspect_term_"+str(xx)]
        if actual_term != None:
            for asp_part in nltk.word_tokenize(actual_term):
                if asp_part not in tokens and asp_part+"-" not in tokens:
                    print(ii,"-",xx,":",tokens)
                    print(actual_term, asp_part)

280 - 1 : ['It', "'s", 'so', 'quick', 'and', 'responsive', 'that', 'it', 'makes', 'working/surfing', 'on', 'a', 'computer', 'so', 'much', 'more', 'pleasurable', '!']
working working
280 - 2 : ['It', "'s", 'so', 'quick', 'and', 'responsive', 'that', 'it', 'makes', 'working/surfing', 'on', 'a', 'computer', 'so', 'much', 'more', 'pleasurable', '!']
surfing surfing
338 - 1 : ['And', 'mine', 'had', 'broke', 'but', 'I', 'sent', 'it', 'in', 'under', 'warranty-no', 'problems', '.']
warranty warranty
350 - 1 : ['In', 'fact', 'I', 'still', 'use', 'manyLegacy', 'programs', '(', 'Appleworks', ',', 'FileMaker', 'Pro', ',', 'Quicken', ',', 'Photoshop', 'etc', ')', '!']
Legacy programs Legacy
522 - 4 : ['fast', ',', 'great', 'screen', ',', 'beautiful', 'apps', 'for', 'a', 'laptop', ';', 'priced', 'at', '1100', 'on', 'the', 'apple', 'website', ';', 'amazon', 'had', 'it', 'for', '1098+', 'tax', '-', 'plus', 'i', 'had', 'a', '10', '%', 'off', 'coupon', 'from', 'amazon-cost', 'me', '998', 'plus', 'tax-',

# Drop lines without aspects

In [20]:
def pol_to_no(sentiment):
  
    if sentiment == "positive":
        pol = 1
    elif sentiment == "negative":
        pol = -1
    elif sentiment == "neutral":
        pol = 0

    return pol

In [21]:
all_pols = []
no_pol = []
for line in df.index:
    pols = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            pols += [pol_to_no(df.loc[line,col])]
    if len(pols) > 0:
        all_pols += [pols]
    else:
        no_pol += [line]

df_pol = pd.DataFrame(data=all_pols)

In [22]:
df.drop(no_pol, axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6
0,892:1,"Boot time is super fast, around anywhere from ...",Boot time,positive,0,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1144:1,tech support would not fix the problem unless ...,tech support,negative,0,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,359:1,Set up was easy.,Set up,positive,0,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,562:1,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,22,31,touchscreen functions,negative,36,57,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,958:1,Other than not being a fan of click pads (indu...,internal speakers,negative,86,103,price tag,positive,203,212,click pads,negative,30,40,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,932:1,also...- excellent operating system- size and ...,operating system,positive,19,35,size,positive,37,41,weight,positive,46,52,mobility,positive,65,73,durability of the battery,positive,85,110,functions provided by the trackpad,positive,116,150
407,256:1,This hardware seems to be better than the iMac...,hardware,positive,5,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
408,520:1,I've had it for about 2 months now and found n...,software,neutral,60,68,updates,neutral,72,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
409,306:2,the latest version does not have a disc drive.,disc drive,neutral,35,45,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# Descriptive Analysis


## Aspects per sentence

In [23]:
pol_cols = ["aspect_polarity_"+str(ii) for ii in range(1,aspect_number)]

In [24]:
prev = 0
total_asp = 0
for no, col in enumerate(pol_cols):
    #print("sentences with at least ", no+1, "aspects:", sum(df[col].value_counts()))

    if col != "aspect_polarity_1":
        print("sentences with exactly ", no, "aspects:", prev - sum(df[col].value_counts()))
        total_asp += no * (prev - sum(df[col].value_counts()))

    if col == "aspect_polarity_"+str(aspect_number-1):
        print("sentences with exactly ", no+1, "aspects:", sum(df[col].value_counts()))
        total_asp += (no+1) * sum(df[col].value_counts())


    prev = sum(df[col].value_counts())

print("total no of aspects: ", total_asp)

sentences with exactly  1 aspects: 258
sentences with exactly  2 aspects: 101
sentences with exactly  3 aspects: 34
sentences with exactly  4 aspects: 10
sentences with exactly  5 aspects: 6
sentences with exactly  6 aspects: 1
total no of aspects:  638


## Sentiment Frequency

In [25]:
df_pol = df.loc[:,pol_cols]
df_pol_counts = df_pol.apply(pd.Series.value_counts)
df_pol_counts.sum(axis=1)

negative    128.0
neutral     169.0
positive    341.0
dtype: float64

In [26]:
sum(df_pol_counts.sum(axis=1))

638.0

## Sentences with more than one aspect

In [27]:
multi_counter = 0
for line in df.index:
    sentiment_list = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            sentiment_list += [df.loc[line,col]]
    if len(set(sentiment_list)) > 1:
        multi_counter += 1
multi_counter

38

# Save as xml

In [ ]:
root = ET.Element('sentences')

for line in df.index:
    name = "sentence"
    entry = ET.SubElement(root, name)
    entry.set("id", str(df["id"][line]))

    text_child = ET.SubElement(entry, "text")
    text_child.text = str(df["text"][line])

    asp_child = ET.SubElement(entry, "aspectTerms")
    for xx in range(1,aspect_number):
        if df.loc[line,"aspect_term_"+str(xx)] != None:
            asp_subchild = ET.SubElement(asp_child, "aspectTerm")
            asp_subchild.set("from",str(df["aspect_from_"+str(xx)][line]))
            asp_subchild.set("polarity",str(df["aspect_polarity_"+str(xx)][line]))
            asp_subchild.set("term",str(df["aspect_term_"+str(xx)][line]))
            asp_subchild.set("to",str(df["aspect_to_"+str(xx)][line]))

xml_data = ET.tostring(root)

In [ ]:
with open('/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/test.xml', 'w') as f:  # Write in file as utf-8
    f.write(xml_data.decode('utf-8'))

# Create xml.seg

In [ ]:
def xml_seg_maker(df):

    df_wo_id = df.drop(columns="id", axis=1, inplace=False)
    df_wo_id.reset_index(drop=True, inplace=True)
    data_lines = []

    for ii in df_wo_id.index:

        line = list(df_wo_id.loc[ii])
        o_text = line[0]
        aspects = [line[xx] for xx in range(1,len(df_wo_id.loc[0]),4) if line[xx] != None]
        pols = [line[xx] for xx in range(2,len(df_wo_id.loc[0]),4) if line[xx] != None]

        for asp in range(len(aspects)):
            text = o_text.replace(aspects[asp],'$T$')
            pol = str(pol_to_no(pols[asp]))

            data_lines += [text,aspects[asp],pol]
    
    return data_lines

In [ ]:
xml_seg_data = xml_seg_maker(df)

with open('/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/test.xml.seg', 'w') as f:
    f.write('\n'.join(xml_seg_data))

# Create BERT+txt

In [ ]:
asp_cols = ["aspect_term_"+str(ii) for ii in range(1,aspect_number)]

In [ ]:
def sent_conv(sentiment):

    if sentiment == "positive":
        return "POS"
    elif sentiment == "negative":
        return "NEG"
    elif sentiment == "neutral":
        return "NEU"

In [ ]:
def txt_maker(df):

    data_lines = []

    for line in df.index:

        text = df.loc[line,"text"]        
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = sent_conv(df.loc[line,pol_cols[col]])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())


        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + "=T-" + asp_sent_dict[tok] + " "
            else:
                label += tok + "=O "
        label = label[:-1]

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    old_label = " ".join([tokens[no+xx]+"=O" for xx in range(ii) if no+xx < len(tokens)])
                    new_label = " ".join([tokens[no+xx]+"=T-"+new_pol for xx in range(ii) if no+xx < len(tokens)])
                    label = label.replace(old_label, new_label)
  
        data_lines += [text+"####"+label]

    return data_lines

In [ ]:
txt_data = txt_maker(df)

with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/test.txt","w") as f:
    f.write('\n'.join(txt_data))

# Create RGATjson

In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 6.9MB/s 


In [ ]:
import stanza
from more_itertools import locate
import json

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,depparse')

2021-05-11 07:03:21 INFO: Downloading default packages for language: en (English)...
2021-05-11 07:05:12 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-05-11 07:05:12 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-05-11 07:05:13 INFO: Use device: cpu
2021-05-11 07:05:13 INFO: Loading: tokenize
2021-05-11 07:05:13 INFO: Loading: pos
2021-05-11 07:05:13 INFO: Loading: lemma
2021-05-11 07:05:13 INFO: Loading: depparse
2021-05-11 07:05:13 INFO: Done loading processors!


## in case of creating for the first time

In [ ]:
def json_make_pos(df):
   
    manual_pos = {}

    for ii in df.index:
        
        new_dict = {}
        text = df.loc[ii,"text"]
        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        
        new_dict["aspects"] = []
        for xx in range(1,aspect_number):         
            if df.loc[ii,"aspect_term_"+str(xx)] != None:
                asp_dict = {}   
                term = df.loc[ii,"aspect_term_"+str(xx)]

                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == tok)) for tok in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:
                    print(ii, ": ", text)
                    print("original term: ",term)
                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    print("all start chars: ",all_char_from)
                    corr_char_from = int(df.loc[ii, "aspect_from_"+str(xx)])
                    print("correct start char: ",corr_char_from)
                    print("text beginning at correct start char: ", text[corr_char_from:])
                    print("tokens: ",tokens)
                    print("original asp tokens: ", asp_toks)
                    print("original asp indices: ",asp_ind)
                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:

                    print("Tokenization Error in line ",ii,"!")
                    for pos, tok in enumerate(tokens):
                        print(pos, tok)
                    print("aspect term: ", term)
                    print("original asp tokens: ", asp_toks)

                    from_index = int(input("start position?"))
                    to_index = int(input("end position?"))

                    # add manually stated positions to dict for reproducibility
                    manual_pos[text] = {}
                    manual_pos[text][term] = {}
                    manual_pos[text][term]["from"] = from_index
                    manual_pos[text][term]["to"] = to_index
                    print(manual_pos)

                    print("Final aspect tokens: ", tokens[from_index:to_index+1])

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

    with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/aspect_positions/semeval_lap_test.json","w") as f:
        json.dump(manual_pos, f)

json_make_pos(df)

Tokenization Error in line  20 !
0 And
1 I
2 may
3 be
4 the
5 only
6 one
7 but
8 I
9 am
10 really
11 liking
12 Windows
13 8.
aspect term:  Windows 8
original asp tokens:  ['Windows', '8']
start position?12
end position?13
{'And I may be the only one but I am really liking Windows 8.': {'Windows 8': {'from': 12, 'to': 13}}}
Final aspect tokens:  ['Windows', '8.']
Tokenization Error in line  53 !
0 I
1 opted
2 for
3 the
4 SquareTrade
5 3
6 -
7 Year
8 Computer
9 Accidental
10 Protection
11 Warranty
12 (
13 $
14 1500
15 -
16 2000
17 )
18 which
19 also
20 support
21 "
22 accidents
23 "
24 like
25 drops
26 and
27 spills
28 that
29 are
30 NOT
31 covered
32 by
33 AppleCare
34 .
aspect term:  SquareTrade 3-Year Computer Accidental Protection Warranty
original asp tokens:  ['Square', 'Trade', '3', '-', 'Year', 'Computer', 'Accidental', 'Protection', 'Warranty']
start position?4
end position?11
{'And I may be the only one but I am really liking Windows 8.': {'Windows 8': {'from': 12, 'to': 13}}, 

## in case of reproducing the dataset

In [ ]:
def json_maker(df, manual_pos):
   
    new_data = []

    for ii in df.index:
        
        new_dict = {}
        text = df.loc[ii,"text"]

        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        new_dict["token"] = tokens

        new_dict["pos"] = [word.xpos for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["head"] = [str(word.head) for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["deprel"] = [word.deprel for sentence in nlp(text).sentences for word in sentence.words]
        
        new_dict["aspects"] = []
        for xx in range(1,aspect_number):         
            if df.loc[ii,"aspect_term_"+str(xx)] != None:
                asp_dict = {}   
                term = df.loc[ii,"aspect_term_"+str(xx)]
                asp_dict["term"] = term
                asp_dict["polarity"] = df.loc[ii,"aspect_polarity_"+str(xx)]

                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == term)) for term in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:

                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    corr_char_from = int(df.loc[ii, "aspect_from_"+str(xx)])

                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:
                    if text in manual_pos.keys() and term in manual_pos[text].keys():
                        from_index = manual_pos[text][term]["from"]
                        to_index = manual_pos[text][term]["to"]

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

                new_dict["aspects"] += [asp_dict]
        
        new_data += [new_dict]

    return new_data

In [ ]:
pos_obj = open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/aspect_positions/semeval_lap_test.json")
loaded_pos = json.load(pos_obj)

json_data = json_maker(df, loaded_pos)

with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/test.json","w") as f:
    json.dump(json_data, f)

# Create LCF-ATEPCdat

In [ ]:
def pol_to_no_shifted(sentiment):
  
    if sentiment == "positive":
        pol = 2
    elif sentiment == "negative":
        pol = 0
    elif sentiment == "neutral":
        pol = 1

    return str(pol)

In [ ]:
def dat_maker(df):
    
    data_lines = []

    for line in df.index:

        text = df.loc[line,"text"]
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = pol_to_no_shifted(df.loc[line,pol_cols[col]])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + " B-ASP -1\n"
            else:
                label += tok + " O -1\n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    label = label.replace(tokens[no]+" O -1",tokens[no]+" B-ASP -1")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx]+" O -1",tokens[no+xx]+" I-ASP -1")

        # create duplicates of review in case of more than one aspect
        for key, val in asp_sent_dict.items():
            if key in tokens:
                new_label = label.replace(key+" B-ASP -1", key+" B-ASP "+val)
                data_lines += [new_label]
                data_lines += ["\n"]
            else:
                for ii in range(2,max_asp_len+1):
                    for no,tok in enumerate(tokens):
                        new_tok = " ".join(tokens[no:no+ii])
                        if new_tok == key:
                            new_label = label.replace(tokens[no]+" B-ASP -1",tokens[no]+" B-ASP "+val)
                            for xx in range(1,ii):
                                new_label = new_label.replace(tokens[no+xx]+" I-ASP -1",tokens[no+xx]+" I-ASP "+val)

                            data_lines += [new_label]
                            data_lines += ["\n"]

    return data_lines

In [ ]:
dat_data = dat_maker(df)

with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/test.dat","w") as f:
    f.write(''.join(dat_data))

# Create GRACEtxt

In [ ]:
asp_cols = ["aspect_term_"+str(ii) for ii in range(1,aspect_number)]

In [ ]:
import nltk

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Create chunks and pos tags. Source: https://towardsdatascience.com/chunking-in-nlp-decoded-b4a71b2b4e24


In [ ]:
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI
from nltk.chunk.util import tree2conlltags, conlltags2tree
from nltk.corpus import conll2000

In [ ]:
def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
    
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

class NGramTagChunker(ChunkParserI):

    def __init__(self,train_sentences,tagger_classes=[UnigramTagger,BigramTagger]):
        train_sent_tags=conll_tag_chunks(train_sentences)
        self.chunk_tagger=combined_tagger(train_sent_tags,tagger_classes)
    
    def parse(self,tagged_sentence):
        if not tagged_sentence:
            return None
        pos_tags=[tag for word, tag in tagged_sentence]
        chunk_pos_tags=self.chunk_tagger.tag(pos_tags)
        chunk_tags=[chunk_tag for (pos_tag,chunk_tag) in chunk_pos_tags]
        wpc_tags=[(word,pos_tag,chunk_tag) for ((word,pos_tag),chunk_tag) in zip(tagged_sentence,chunk_tags)]
        return conlltags2tree(wpc_tags)

In [ ]:
data = conll2000.chunked_sents()
ntc = NGramTagChunker(data)

Convert tags into BIOES scheme. Source: https://gist.github.com/allanj/5ad206f7f4645c0269b68fb2065712f4

In [ ]:
def iob_iobes(tags):
    """
    IOB2 (BIO) -> IOBES
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag == 'O':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'B':
            if i + 1 != len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('B-', 'S-'))
        elif tag.split('-')[0] == 'I':
            if i + 1 < len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('I-', 'E-'))
        else:
            raise Exception('Invalid IOB format!')
    return new_tags

In [ ]:
def grace_txt_maker(df):
    
    data_lines = []

    for line in df.index:

        text = df.text[line]
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create pos tags
        pos_tags = nltk.pos_tag(tokens)

        # create chunk/phrase tags
        full_tags = tree2conlltags(ntc.parse(pos_tags))
        chunks_list = [full_tags[ii][2] for ii in range(len(full_tags))]
        new_chunks = iob_iobes(chunks_list)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = df.loc[line,pol_cols[col]].upper()
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for pos,tok in enumerate(tokens):
            label += tok + " " + pos_tags[pos][1] + " " + new_chunks[pos]
            if tok in asp_sent_dict.keys():
                label +=  " B_AP " + asp_sent_dict[tok] + " B_AP+" + asp_sent_dict[tok] + "\n"
            else:
                label += " O O O \n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    label = label.replace(tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " O O O \n",
                                          tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " B_AP " + \
                                          new_pol + " B_AP+" + new_pol + "\n")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx] + " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " O O O \n",
                                          tokens[no+xx]+ " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " I_AP " + \
                                          new_pol + " I_AP+" + new_pol + "\n")

        data_lines += [label]

    return data_lines

In [ ]:
grace_txt_data = grace_txt_maker(df)

with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/grace_test.txt","w") as f:
    f.write('\n'.join(grace_txt_data))